In [ ]:
import requests

url = 'https://api.bilibili.com/pgc/view/web/season?season_id=33378'
res = requests.get(url, headers={'User-Agent':'Chrome/87.0.4280.88'})

with open('tmp_playlist.json', 'w', encoding='utf8') as f:
    f.write(res.text)

In [ ]:
import json

with open('tmp_playlist.json', 'r', encoding='utf8') as f:
    data = json.load(f)

assert data['code'] == 0
assert data['message'] == 'success'

videos = []

for ep in data['result']['episodes']:
    x = {}
    videos.append(x)

    x['playId'] = 'ep' + str(ep['id'])
    x['tvid'] = int(ep['title'])
    x['title'] = ep['long_title']
    x['pub'] = ep['pub_time']

len(videos)
videos[3]

with open('tvlist.json', 'w', encoding='utf8') as f:
    json.dump(videos, f, ensure_ascii=False, indent=4)

In [ ]:
# zh.wikipedia.org/wiki/名偵探柯南動畫集數列表

from bs4 import BeautifulSoup
from typing import List
from itertools import islice
import re

s = BeautifulSoup(open('wiki-zh.html', 'r', encoding='utf8'))

def extract(table, cols:List[int], skip_rows=2) -> List:
    info_list = []
    for tr in islice(table.select('tr'), skip_rows, None):
        tds = tr.select('td')
        info = []
        for i in cols:
            info.append(tds[i].text)
        info_list.append(info)
    return info_list

table_count = 13

col_list = [3] * table_count
col_list[0] = 4
col_list = [[0, p] for p in col_list]

tables = s.select('.wikitable')
len(tables)

records = []
for i in range(table_count):
    x = extract(tables[i], col_list[i])
    records += x

# process records, remove mark like [2]
for r in records:
    r[1] = re.sub(r'\[\w+?\]', '', r[1])

with open('wiki-result-zh.txt', 'w', encoding='utf8') as f:
    for r in records:
        f.write(f'{r[0]} {r[1]}')
        f.write('\n')